# High order statistic codes

Notebook for reading weak lensing maps maps and to run High order statistics code
All maps and codes are produced by the DESC-HOS topical team (https://github.com/LSSTDESC/HOS-Y1-prep)

- <font color='green'>map2alm </font>
- <font color='green'>Map3</font>
- <font color='green'>PDF_Peaks_Minima</font> -> runs in smoothed map
- integrated3PCF -> needs shear catalogues
- DSS -> needs foreground catalogues

In [1]:
cd ~/HOS-Y1-prep/hoscodes/

/global/u2/j/jatorres/HOS-Y1-prep/hoscodes


In [2]:
import numpy as np 
import matplotlib.pyplot as plt

In [3]:
import healsparse
import random
import os
from HOScodes import *

In [10]:
#map2 nside
nside_c = 32
#thetas=[4,8,16,32] # Aperture scale radii in arcmin
nshells = [19,50,106]
nside= 4096
#tomobins=[0,1,2,3,4] # Tomographic bins
snapshots = [[8,12,15,16],
           [23,32,42,44],
          [34,58,84,90]]
nzs = 'kappa_zsnapshots'
seed = 0 


In [11]:
#Path to maps
path = '/global/cscratch1/sd/xuod/HOS_sims/L845/HACC150/'

kappa_bins_n19 = []
kappa_bins_n50 = []
kappa_bins_n106 = []
for s in range(4):
    m1 = path+'shells_z'+str(nshells[0])+'_subsampleauto_groupiso/'+nzs+'/kappa_hacc_nz'+str(snapshots[0][s])+'_nside4096_seed'+str(seed)+'.fits'
    m2 = path+'shells_z'+str(nshells[1])+'_subsampleauto_groupiso/'+nzs+'/kappa_hacc_nz'+str(snapshots[1][s])+'_nside4096_seed'+str(seed)+'.fits'
    m3 = path+'shells_z'+str(nshells[2])+'_subsampleauto_groupiso/'+nzs+'/kappa_hacc_nz'+str(snapshots[1][s])+'_nside4096_seed'+str(seed)+'.fits'
    kappa_bins_n19.append(m1)
    kappa_bins_n50.append(m2)
    kappa_bins_n106.append(m3)

In [12]:
kappa_maps_n19 = kappamaps(kappa_bins_n19,nshells[0],seed,nzs,nside)
#kappa_maps_n50 = M.readmaps(kappa_bins_n50)
#kappa_maps_n106 = M.readmaps(kappa_bins_n106)

In [13]:
#read maps using the HOScodes.readkappamaps class and start the hoscode class loading the map
results = hoscodes(kappa_bins_n19,nshells[0],seed,nzs,nside)#We save the results here

In [14]:
results.readmaps()

In [15]:
#for run individual codes in a specific tomo bin (bins [0,5] -> in name Ntomo [1,6])
map2alm_bin1 = results.run_map2alm(results.mapbins[0],Ntomo=1)#runs for input

In [ ]:
map3alm_bin1 = results.run_map3(results.mapbins[0],Ntomo=1,theta=[4])#it takes time

In [ ]:
ls results/